In [1]:
import pathpy as pp
from pathpy import Network
import matplotlib.pyplot as plt
from typing import TYPE_CHECKING, Dict, List, Tuple, Union, Any
import scipy.sparse
from scipy.sparse import linalg as spl
import numpy as np


from collections import defaultdict # think these two are used to iterate through dicts
from collections.abc import Iterable
from __future__ import annotations
from typing import TYPE_CHECKING, Union, Dict, Tuple

os.chdir("..")

pathpy tutorials: https://github.com/pathpy/pathpy-tutorials/tree/master/code
too much hassle to figure out everything that's changed


# Core-periphery for $\mathcal{M}_1$ network
For directed and weighted networks. But needs to be connected because uses stationary distributions. For now take reduced connected network and look at teleportation later if possible. <br>
Node strength $s_i$ is the sum of the weights of the links connected to that node
1. Start at the node with the minimum total node strength $s^{in}_i+s^{out}_i$ and set $S=v_i$
2. Add a node to $S$ such that it creates the smallest increase in persistence probability \begin{equation}
\alpha_S = \frac{\sum_{i,j \in S}p_i^*T_{ij} }{\sum_{i \in S}p_i^*}
\end{equation}
3. Then the $\alpha$-periphery is the set of nodes $S_{\alpha}$ satisfying $\alpha_i \leq \alpha$ and you can tune by $\alpha$
Choose uniformly if you get have multiple nodes with the same strength etc

In [24]:
# from paths dataset - 1% sample to start
flight_paths = pp.io.csv.read_pathcollection("Data/US flights 2011/US flights od.ngram", separator=',', frequency=True)

In [30]:
pp.core.path.nodes(flight_paths)

AttributeError: module 'pathpy.core.path' has no attribute 'nodes'

In [31]:
net = pp.core.network.Network(flight_paths)

In [32]:
net

ValueError: max() arg is an empty sequence

Network {<pathpy.core.path.Path object at 0x1731eb820>, <pathpy.core.path.Path object at 0x173211820>, <pathpy.core.path.Path object at 0x173280820>, <pathpy.core.path.Path object at 0x17329a820>, <pathpy.core.path.Path object at 0x1732b5820>, <pathpy.core.path.Path object at 0x173236850>, <pathpy.core.path.Path object at 0x17324f850>, <pathpy.core.path.Path object at 0x1732a6850>, <pathpy.core.path.Path object at 0x173242070>, <pathpy.core.path.Path object at 0x17328d070>, <pathpy.core.path.Path object at 0x17329a070>, <pathpy.core.path.Path object at 0x1732b5070>, <pathpy.core.path.Path object at 0x1731f3880>, <pathpy.core.path.Path object at 0x1731fc880>, <pathpy.core.path.Path object at 0x173207880>, <pathpy.core.path.Path object at 0x17329a0a0>, <pathpy.core.path.Path object at 0x1732070a0>, <pathpy.core.path.Path object at 0x1732680a0>, <pathpy.core.path.Path object at 0x17328d8b0>, <pathpy.core.path.Path object at 0x1731dc8b0>, <pathpy.core.path.Path object at 0x17328d0d0>, <pat

In [21]:
help(pp.core.network)

Help on module pathpy.core.network in pathpy.core:

NAME
    pathpy.core.network - Network class

CLASSES
    pathpy.core.base.classes.BaseModel(pathpy.core.base.classes.AbstractNetwork, pathpy.core.base.classes.BaseClass)
        Network
    
    class Network(pathpy.core.base.classes.BaseModel)
     |  Network(uid: 'Optional[str]' = None, directed: 'bool' = True, multiedges: 'bool' = False, **kwargs: 'Any') -> 'None'
     |  
     |  Class for a network.
     |  
     |  A network is a structure amounting to a set of objects in which some of the
     |  objects are related to each other. The objects correspond to mathematical
     |  abstractions called nodes (or vertices) and each of the related pairs of
     |  nodes is called an edge (or link). Furthermore, related edges form
     |  paths. Thereby, the edges and paths may be directed or undirected.
     |  
     |  In ``pathpy`` a :py:class:`Network` stores :py:class:`Node`,
     |  :py:class:`Edge` and :py:class:`Path` objects w

In [12]:

flight_paths = pp.Paths.read_file("Data/US flights 2011/US flights od.ngram", separator=',', frequency=True)
m1 = pp.Network.from_paths(flight_paths)

AttributeError: module 'pathpy' has no attribute 'Paths'

In [6]:
# need a dict with indexes and need strengths
indegrees = m1.degrees(mode='indegree')
outdegrees= m1.degrees(mode='outdegree')


In [34]:
# from networks module in pathpy this has weights

def degrees(self, weight: Weight = None) -> Dict[str, float]:
    """Retuns a dict with degrees of the nodes."""
    if weight is None:
        _d = self._degrees(self._properties['degrees'], weight)
    else:
        _d = self._degrees(self._properties['incident_edges'], weight)
    return _d



In [4]:
m1.degrees_sequence()

NameError: name 'm1' is not defined

In [33]:
help(m1.degrees())

Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

In [25]:
degree_sequence(m1,weight=True)

AttributeError: 'collections.defaultdict' object has no attribute 'uids'

In [30]:
help(pp.network)

Help on module pathpy.classes.network in pathpy.classes:

NAME
    pathpy.classes.network - # -*- coding: utf-8 -*-

CLASSES
    builtins.dict(builtins.object)
        UnorderedDict
    builtins.object
        Network
    
    class Network(builtins.object)
     |  Network(directed=False)
     |  
     |  A graph or network that can be directed, undirected, unweighted or weighted
     |  and whose edges can contain arbitrary attributes. This is the base class for
     |  HigherOrderNetwork
     |  
     |  Attributes
     |  ----------
     |  
     |  nodes : list
     |      A list of (string) nodes.
     |  edges : dictionary
     |      A dictionary containing edges (as tuple-valued keys) and their attributes (as value)
     |  
     |  Methods defined here:
     |  
     |  __add__(self, other)
     |      Add two networks and return the union of both
     |      
     |      Parameters
     |      ----------
     |      other : Network
     |      
     |      Returns
     |     

# Centrality functions

In [2]:
## from pathpy repo

def eigenvector_centrality(network: Network, # arg name and type (didn't know you could do this)
                           weight: Union[str, bool, None] = None,
                           alpha=0.85,
                           **kwargs: Any) -> dict:
    """Calculates the eigenvector centrality of all nodes.

    Parameters
    ----------
    network : Network

        The :py:class:`Network` object that contains the network

    Examples
    --------
    Compute eigenvector centrality in a simple network

    >>> import pathpy as pp
    >>> net = pp.Network(directed=True)
    >>> net.add_edge('a', 'x')
    >>> net.add_edge('x', 'b')
    >>> c = pp.algorithms.centralities.eigenvector_centrality(net)
    >>> c['a']
    1
    """
    evcent: dict = dict()
        
    A = network.adjacency_matrix(weighted=True, transposed=True) # changed this for my version
    N = A.shape[0] # total nodes
    I = scipy.sparse.identity(N) # identity
    ev = spl.inv(I - alpha*A).dot(ones)
    
    if kwargs:
        _, ev = spl.eigs(A, k=1, which='LM', **kwargs) # which='LM' means find biggest, kwargs if changing eigenvec to test
    else:
        _, ev = spl.eigs(A, k=1, which='LM') # only returns vec for stationary distributino
    ev = ev.reshape(ev.size, ) # makes it a column vector I think
    S = np.sum(ev)
    
    for v, deg in network.nodes.items():
        evcent[v] = np.real(ev[network.nodes.index[v.uid]]/S) 
    for v in network.nodes:
        evcent[v.uid] = np.real(ev[network.nodes.index[v.uid]]/S) 
    return evcent


# so last few lines: take eigenvector corresponding to largest eigenvalue (1) of adjacency matrix of net
# reshape this vector to a column(?) vector. evcent is an empty dictionary. Then define for each key 1.uid 
# (not sure what uid stands for...) the values is the real part of the stationary probability averaged over 
# all the stationary probabilities and these will always sum to 1

In [3]:
# apparently this has ill-defined transition probability at b so I need teleportation
net = pp.Network(directed=True)
net.add_edge('a', 'x')
net.add_edge('x', 'b')
net.add_edge('a','c')
net.add_edge('x','b')
net.add_edge('c','x')
network = net

## Trying to come generalise PageRank like in $(181)$

\begin{equation}
p^* = \frac{1-\alpha}{N}\begin{pmatrix}1 & ... & 1 \end{pmatrix} \begin{pmatrix} I - \alpha T \end{pmatrix}^{-1}
\end{equation}

My main issues are keeping it efficient, use scipy.sparse etc. and also keeping track of the indices...except now that I think about it all that matters are the indices of $p^*=(p_1^*...p_N^*)$ so it's ok. <br><br>
T is defined wrong here, it is meant to be $T_{ij} = \frac{A_{ij}}{k_i^{\text{out}}}$., just going to assume $A_{ij}$ defines a path from $i \to j$ but might need to change this. Hence each row will need to be divide by it's sum to get $T$.

In [169]:

def my_pagerank(network,alpha = 0.85):

    node_names = list(network.nodes.keys())

    A = network.adjacency_matrix(weighted=True, transposed=False) # unsure about transpose
    N = A.shape[0] # total nodes
    I = scipy.sparse.identity(N) # identity
    ones = np.ones(N)
    Ap = spl.inv(I - alpha*A)
    ev = (1-alpha)*Ap.dot(ones)/N
    S = sum(ev)
    ev = ev/S # normalised for some reason - might not be correct I need to check
    evcents = list(zip(node_names, zip(ev)))
    
    return evcents

I should be looking at PageRank because that's for directed networks (https://stats.stackexchange.com/questions/176874/pagerank-vs-eigenvector-centrality) PageRank uses in-degree of nodes specifically so is a special case of eigenvector centrality.

In [168]:
A = network.adjacency_matrix(weighted=True, transposed=False) # unsure about transpose
print(A)

  (0, 1)	1.0
  (0, 3)	1.0
  (1, 2)	1.0
  (3, 1)	1.0


In [197]:
out_deg = A.sum(1)
out_deg = np.squeeze(np.asarray(out_deg))

In [200]:
print(A)

  (0, 1)	1.0
  (0, 3)	1.0
  (1, 2)	1.0
  (3, 1)	1.0


In [201]:
out_deg = scipy.sparse.spdiags(out_deg, 0, out_deg.size, out_deg.size)
out_deg.dot(A)

/usr/local/lib/python3.9/site-packages/scipy/sparse/sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


TypeError: no supported conversion for types: (dtype('O'),)

In [165]:
network

In [164]:
my_pagerank(network)

[('a', (0.4674229310154322,)),
 ('x', (0.1816998760021117,)),
 ('b', (0.09821614919033063,)),
 ('c', (0.25266104379212556,))]

In [160]:
pp.algorithms.centralities.pagerank(network)

{'a': 0.12045209234069676,
 'x': 0.3175410567582492,
 'b': 0.39036291569760717,
 'c': 0.1716439352034471}

My calculated results are different to the results of the PageRank function, but which is more accurate? <br>
Neither perform very well tbh. It's clear that b's centrality is too high because it has no out-degree ( can't see function so don't know if that transition matrix has been set up yet, I must fix that actually).

In [162]:
network

In [94]:
A = net.adjacency_matrix()
print(A)

  (0, 1)	1.0
  (1, 2)	1.0


In [19]:
at = a.reshape(a.size,)
at

array([ 0.00000000e+000, -2.68678569e+154,  1.48219694e-323,
        0.00000000e+000,  0.00000000e+000,  4.17201348e-309])

In [20]:
at.shape

(6,)